In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
def create_convnext_like_model(input_dim, num_classes, sliding_window_size=3):
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Reshape input to be compatible with Conv1D using Lambda layer
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)  # Shape: (batch_size, input_dim, 1)
    x = reshaped_input
    
    # Simulate ConvNeXt block using Conv1D layers
    x = Conv1D(filters=int(max(16, input_dim * 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 4)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    
    # Flatten the output of the final Conv1D layer
    x = Flatten()(x)
    
    # Global pooling and classification head
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        model = create_convnext_like_model(train_X.shape[1], test_y.shape[1])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            train_X, train_y, 
            epochs=100, 
            batch_size=int(train_X.shape[0] * 0.01),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [ ]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/50 started...

Aggregated Metrics for 1 runs:
test_loss: 0.0012
test_accuracy: 0.9996
balanced_accuracy: 0.9912
roc_auc: 1.0000
pr_auc: 1.0000
f2: 0.9996
mcc: 0.9988
0_precision: 0.9998
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 9117.0000
1_precision: 1.0000
1_recall: 1.0000
1_f1-score: 1.0000
1_support: 10.0000
2_precision: 0.9143
2_recall: 0.9412
2_f1-score: 0.9275
2_support: 34.0000
3_precision: 1.0000
3_recall: 0.9989
3_f1-score: 0.9994
3_support: 1781.0000
4_precision: 1.0000
4_recall: 0.9985
4_f1-score: 0.9992
4_support: 653.0000
5_precision: 1.0000
5_recall: 1.0000
5_f1-score: 1.0000
5_support: 2.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 3.0000
accuracy: 0.9996
macro avg_precision: 0.9877
macro avg_recall: 0.9912
macro avg_f1-score: 0.9894
macro avg_support: 11600.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0.9996
weighted avg_f1-score: 0.9996
weighted avg_support: 11600.0000
Run 2/50 started...

Aggregated Metrics for 2 runs:

In [ ]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/10 started...

Aggregated Metrics for 1 runs:
test_loss: 0.2359
test_accuracy: 0.9051
balanced_accuracy: 0.8268
roc_auc: 0.9921
pr_auc: 0.9675
f2: 0.9048
mcc: 0.8470
0_precision: 0.9171
0_recall: 0.8871
0_f1-score: 0.9019
0_support: 42368.0000
1_precision: 0.9052
1_recall: 0.9366
1_f1-score: 0.9206
1_support: 56661.0000
2_precision: 0.8716
2_recall: 0.9141
2_f1-score: 0.8924
2_support: 7151.0000
3_precision: 0.8830
3_recall: 0.7286
3_f1-score: 0.7984
3_support: 549.0000
4_precision: 0.8295
4_recall: 0.6609
4_f1-score: 0.7356
4_support: 1899.0000
5_precision: 0.8116
5_recall: 0.7676
5_f1-score: 0.7890
5_support: 3473.0000
6_precision: 0.9529
6_recall: 0.8925
6_f1-score: 0.9217
6_support: 4102.0000
accuracy: 0.9051
macro avg_precision: 0.8815
macro avg_recall: 0.8268
macro avg_f1-score: 0.8514
macro avg_support: 116203.0000
weighted avg_precision: 0.9050
weighted avg_recall: 0.9051
weighted avg_f1-score: 0.9046
weighted avg_support: 116203.0000
Run 2/10 started...

Aggregated Metri

In [ ]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("kdd_train.csv", "kdd_test.csv", is_string_labels = True, label_mapping=labels_map)

Memory usage of dataframe is 789.51 MB
Memory usage after optimization is: 196.60 MB
Decreased by 75.1%
Memory usage of dataframe is 197.38 MB
Memory usage after optimization is: 49.15 MB
Decreased by 75.1%
Verbose output only for first run...

Run 1/50 started...
Epoch 1/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1209s 12s/step - accuracy: 0.9384 - loss: 0.2227
Epoch 2/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1129s 11s/step - accuracy: 0.9978 - loss: 0.0096
Epoch 3/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1171s 12s/step - accuracy: 0.9984 - loss: 0.0069
Epoch 4/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1131s 11s/step - accuracy: 0.9986 - loss: 0.0056
Epoch 5/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1128s 11s/step - accuracy: 0.9987 - loss: 0.0050
Epoch 6/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1139s 11s/step - accuracy: 0.9989 - loss: 0.0043
Epoch 7/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1196s 12s/step - accuracy: 0.9990 - loss: 0.0040
Epoch 8/100
101/101 ━━━━━━━━━━━━━━━━━━━━ 1127s 11s/step - accuracy: 0.9992 - loss: 0.0033
Epoch 9/100
101

In [ ]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/10 started...



Aggregated Metrics for 1 runs:
test_loss: 0.0028
test_accuracy: 0.9993
balanced_accuracy: 0.9811
roc_auc: 1.0000
pr_auc: 0.9999
f2: 0.9993
mcc: 0.9974
0_precision: 0.9997
0_recall: 1.0000
0_f1-score: 0.9998
0_support: 26862.0000
1_precision: 0.9996
1_recall: 0.9989
1_f1-score: 0.9992
1_support: 2657.0000
2_precision: 0.9967
2_recall: 0.9967
2_f1-score: 0.9967
2_support: 908.0000
3_precision: 0.9943
3_recall: 0.9981
3_f1-score: 0.9962
3_support: 522.0000
4_precision: 0.9932
4_recall: 0.9898
4_f1-score: 0.9915
4_support: 293.0000
5_precision: 0.9925
5_recall: 0.9851
5_f1-score: 0.9888
5_support: 269.0000
6_precision: 0.9912
6_recall: 0.9741
6_f1-score: 0.9826
6_support: 116.0000
7_precision: 0.9592
7_recall: 0.8868
7_f1-score: 0.9216
7_support: 53.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 44.0000
accuracy: 0.9993
macro avg_precision: 0.9918
macro avg_recall: 0.9811
macro avg_f1-score: 0.9863
macro avg_support: 31724.0000
weighted avg_precision: 0.9993
weigh


Aggregated Metrics for 2 runs:
test_loss: 0.0206
test_accuracy: 0.9947
balanced_accuracy: 0.9315
roc_auc: 0.9994
pr_auc: 0.9983
f2: 0.9947
mcc: 0.9808
0_precision: 0.9982
0_recall: 0.9996
0_f1-score: 0.9989
0_support: 26862.0000
1_precision: 0.9950
1_recall: 0.9806
1_f1-score: 0.9877
1_support: 2657.0000
2_precision: 0.9819
2_recall: 0.9851
2_f1-score: 0.9835
2_support: 908.0000
3_precision: 0.9345
3_recall: 0.9674
3_f1-score: 0.9504
3_support: 522.0000
4_precision: 0.9729
4_recall: 0.9420
4_f1-score: 0.9570
4_support: 293.0000
5_precision: 0.8997
5_recall: 0.8885
5_f1-score: 0.8940
5_support: 269.0000
6_precision: 0.9656
6_recall: 0.8922
6_f1-score: 0.9265
6_support: 116.0000
7_precision: 0.8519
7_recall: 0.7736
7_f1-score: 0.8108
7_support: 53.0000
8_precision: 0.8704
8_recall: 0.9545
8_f1-score: 0.9082
8_support: 44.0000
accuracy: 0.9947
macro avg_precision: 0.9411
macro avg_recall: 0.9315
macro avg_f1-score: 0.9352
macro avg_support: 31724.0000
weighted avg_precision: 0.9948
weigh


Aggregated Metrics for 3 runs:
test_loss: 0.0331
test_accuracy: 0.9898
balanced_accuracy: 0.8759
roc_auc: 0.9991
pr_auc: 0.9973
f2: 0.9894
mcc: 0.9621
0_precision: 0.9928
0_recall: 0.9997
0_f1-score: 0.9962
0_support: 26862.0000
1_precision: 0.9946
1_recall: 0.9721
1_f1-score: 0.9832
1_support: 2657.0000
2_precision: 0.9792
2_recall: 0.9714
2_f1-score: 0.9752
2_support: 908.0000
3_precision: 0.9486
3_recall: 0.8876
3_f1-score: 0.9117
3_support: 522.0000
4_precision: 0.9703
4_recall: 0.7224
4_f1-score: 0.7840
4_support: 293.0000
5_precision: 0.8461
5_recall: 0.8414
5_f1-score: 0.8437
5_support: 269.0000
6_precision: 0.9478
6_recall: 0.8937
6_f1-score: 0.9191
6_support: 116.0000
7_precision: 0.8048
7_recall: 0.6855
7_f1-score: 0.7383
7_support: 53.0000
8_precision: 0.8879
8_recall: 0.9091
8_f1-score: 0.8946
8_support: 44.0000
accuracy: 0.9898
macro avg_precision: 0.9302
macro avg_recall: 0.8759
macro avg_f1-score: 0.8940
macro avg_support: 31724.0000
weighted avg_precision: 0.9897
weigh


Aggregated Metrics for 4 runs:
test_loss: 0.0349
test_accuracy: 0.9899
balanced_accuracy: 0.8756
roc_auc: 0.9990
pr_auc: 0.9971
f2: 0.9896
mcc: 0.9625
0_precision: 0.9934
0_recall: 0.9997
0_f1-score: 0.9965
0_support: 26862.0000
1_precision: 0.9948
1_recall: 0.9697
1_f1-score: 0.9820
1_support: 2657.0000
2_precision: 0.9785
2_recall: 0.9656
2_f1-score: 0.9719
2_support: 908.0000
3_precision: 0.9396
3_recall: 0.8951
3_f1-score: 0.9125
3_support: 522.0000
4_precision: 0.9688
4_recall: 0.7713
4_f1-score: 0.8231
4_support: 293.0000
5_precision: 0.8234
5_recall: 0.8346
5_f1-score: 0.8287
5_support: 269.0000
6_precision: 0.9363
6_recall: 0.8879
6_f1-score: 0.9108
6_support: 116.0000
7_precision: 0.7829
7_recall: 0.6698
7_f1-score: 0.7204
7_support: 53.0000
8_precision: 0.8910
8_recall: 0.8864
8_f1-score: 0.8852
8_support: 44.0000
accuracy: 0.9899
macro avg_precision: 0.9232
macro avg_recall: 0.8756
macro avg_f1-score: 0.8924
macro avg_support: 31724.0000
weighted avg_precision: 0.9898
weigh


Aggregated Metrics for 5 runs:
test_loss: 0.0371
test_accuracy: 0.9887
balanced_accuracy: 0.8635
roc_auc: 0.9989
pr_auc: 0.9970
f2: 0.9883
mcc: 0.9581
0_precision: 0.9922
0_recall: 0.9997
0_f1-score: 0.9959
0_support: 26862.0000
1_precision: 0.9938
1_recall: 0.9685
1_f1-score: 0.9810
1_support: 2657.0000
2_precision: 0.9776
2_recall: 0.9617
2_f1-score: 0.9695
2_support: 908.0000
3_precision: 0.9431
3_recall: 0.8785
3_f1-score: 0.9058
3_support: 522.0000
4_precision: 0.9707
4_recall: 0.7106
4_f1-score: 0.7851
4_support: 293.0000
5_precision: 0.8144
5_recall: 0.8141
5_f1-score: 0.8139
5_support: 269.0000
6_precision: 0.9205
6_recall: 0.8966
6_f1-score: 0.9072
6_support: 116.0000
7_precision: 0.7698
7_recall: 0.6604
7_f1-score: 0.7097
7_support: 53.0000
8_precision: 0.8855
8_recall: 0.8818
8_f1-score: 0.8809
8_support: 44.0000
accuracy: 0.9887
macro avg_precision: 0.9186
macro avg_recall: 0.8635
macro avg_f1-score: 0.8832
macro avg_support: 31724.0000
weighted avg_precision: 0.9886
weigh


Aggregated Metrics for 6 runs:
test_loss: 0.0377
test_accuracy: 0.9887
balanced_accuracy: 0.8638
roc_auc: 0.9988
pr_auc: 0.9969
f2: 0.9884
mcc: 0.9584
0_precision: 0.9924
0_recall: 0.9998
0_f1-score: 0.9960
0_support: 26862.0000
1_precision: 0.9939
1_recall: 0.9684
1_f1-score: 0.9810
1_support: 2657.0000
2_precision: 0.9767
2_recall: 0.9578
2_f1-score: 0.9671
2_support: 908.0000
3_precision: 0.9407
3_recall: 0.8774
3_f1-score: 0.9047
3_support: 522.0000
4_precision: 0.9699
4_recall: 0.7366
4_f1-score: 0.8065
4_support: 293.0000
5_precision: 0.8102
5_recall: 0.8104
5_f1-score: 0.8100
5_support: 269.0000
6_precision: 0.9144
6_recall: 0.8894
6_f1-score: 0.9007
6_support: 116.0000
7_precision: 0.7640
7_recall: 0.6635
7_f1-score: 0.7090
7_support: 53.0000
8_precision: 0.8958
8_recall: 0.8712
8_f1-score: 0.8804
8_support: 44.0000
accuracy: 0.9887
macro avg_precision: 0.9175
macro avg_recall: 0.8638
macro avg_f1-score: 0.8839
macro avg_support: 31724.0000
weighted avg_precision: 0.9887
weigh


Aggregated Metrics for 7 runs:
test_loss: 0.0377
test_accuracy: 0.9889
balanced_accuracy: 0.8620
roc_auc: 0.9988
pr_auc: 0.9969
f2: 0.9887
mcc: 0.9591
0_precision: 0.9928
0_recall: 0.9997
0_f1-score: 0.9963
0_support: 26862.0000
1_precision: 0.9934
1_recall: 0.9692
1_f1-score: 0.9811
1_support: 2657.0000
2_precision: 0.9761
2_recall: 0.9588
2_f1-score: 0.9673
2_support: 908.0000
3_precision: 0.9343
3_recall: 0.8799
3_f1-score: 0.9034
3_support: 522.0000
4_precision: 0.9690
4_recall: 0.7591
4_f1-score: 0.8238
4_support: 293.0000
5_precision: 0.8061
5_recall: 0.8163
5_f1-score: 0.8107
5_support: 269.0000
6_precision: 0.9197
6_recall: 0.8596
6_f1-score: 0.8855
6_support: 116.0000
7_precision: 0.7629
7_recall: 0.6523
7_f1-score: 0.7020
7_support: 53.0000
8_precision: 0.8964
8_recall: 0.8636
8_f1-score: 0.8771
8_support: 44.0000
accuracy: 0.9889
macro avg_precision: 0.9167
macro avg_recall: 0.8620
macro avg_f1-score: 0.8830
macro avg_support: 31724.0000
weighted avg_precision: 0.9889
weigh


Aggregated Metrics for 8 runs:
test_loss: 0.0388
test_accuracy: 0.9884
balanced_accuracy: 0.8571
roc_auc: 0.9988
pr_auc: 0.9969
f2: 0.9881
mcc: 0.9572
0_precision: 0.9923
0_recall: 0.9997
0_f1-score: 0.9960
0_support: 26862.0000
1_precision: 0.9936
1_recall: 0.9681
1_f1-score: 0.9807
1_support: 2657.0000
2_precision: 0.9764
2_recall: 0.9581
2_f1-score: 0.9671
2_support: 908.0000
3_precision: 0.9337
3_recall: 0.8803
3_f1-score: 0.9037
3_support: 522.0000
4_precision: 0.9717
4_recall: 0.7116
4_f1-score: 0.7893
4_support: 293.0000
5_precision: 0.8041
5_recall: 0.8123
5_f1-score: 0.8078
5_support: 269.0000
6_precision: 0.9038
6_recall: 0.8675
6_f1-score: 0.8814
6_support: 116.0000
7_precision: 0.7619
7_recall: 0.6580
7_f1-score: 0.7049
7_support: 53.0000
8_precision: 0.9028
8_recall: 0.8580
8_f1-score: 0.8772
8_support: 44.0000
accuracy: 0.9884
macro avg_precision: 0.9156
macro avg_recall: 0.8571
macro avg_f1-score: 0.8787
macro avg_support: 31724.0000
weighted avg_precision: 0.9884
weigh


Aggregated Metrics for 9 runs:
test_loss: 0.0389
test_accuracy: 0.9886
balanced_accuracy: 0.8616
roc_auc: 0.9987
pr_auc: 0.9969
f2: 0.9883
mcc: 0.9579
0_precision: 0.9927
0_recall: 0.9996
0_f1-score: 0.9961
0_support: 26862.0000
1_precision: 0.9928
1_recall: 0.9681
1_f1-score: 0.9802
1_support: 2657.0000
2_precision: 0.9765
2_recall: 0.9578
2_f1-score: 0.9670
2_support: 908.0000
3_precision: 0.9297
3_recall: 0.8851
3_f1-score: 0.9044
3_support: 522.0000
4_precision: 0.9713
4_recall: 0.7353
4_f1-score: 0.8067
4_support: 293.0000
5_precision: 0.8052
5_recall: 0.8104
5_f1-score: 0.8074
5_support: 269.0000
6_precision: 0.9037
6_recall: 0.8688
6_f1-score: 0.8824
6_support: 116.0000
7_precision: 0.7446
7_recall: 0.6688
7_f1-score: 0.7013
7_support: 53.0000
8_precision: 0.9032
8_recall: 0.8611
8_f1-score: 0.8794
8_support: 44.0000
accuracy: 0.9886
macro avg_precision: 0.9133
macro avg_recall: 0.8616
macro avg_f1-score: 0.8805
macro avg_support: 31724.0000
weighted avg_precision: 0.9886
weigh